In [1]:
import pandas as pd
from baselines.shap_c import ShapCounterfactual
from certa.models.utils import get_model
from certa.utils import merge_sources, to_token_df
from certa.local_explain import get_original_prediction, get_row
import os

2023-03-08 09:34:57.275384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mtype = 'deeper'
dataset = 'beers'
base_datadir = '../cheapER/datasets/'
exp_dir = 'experiments/shap_c'

In [3]:
modeldir = 'models/saved/' + mtype + '/' + dataset
model_name = mtype
datadir = base_datadir + dataset
model = get_model(mtype, modeldir, datadir, dataset)

* Costruzione indice degli embeddings.....Fatto. 400001 embeddings totali.
* Creazione del modello per il calcolo degli embeddings....
* Inizializzo il tokenizzatore.....Fatto: 400001 parole totali.
* Preparazione della matrice di embedding.....Fatto. Dimensioni matrice embeddings: (400002, 50)

°°° EMBEDDING MODEL °°°


2023-03-08 09:35:23.711392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Tupla_A (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 Tupla_B (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 Embedding_lookup (Embedding)   (None, None, 50)     20000100    ['Tupla_A[0][0]',                
                                                                  'Tupla_B[0][0]']                
                                                                                                  
Total params: 20,000,100
Trainable params: 0
Non-trainable params: 20,000,100
________________

In [4]:
samples = 10
test = pd.read_csv(datadir + '/test.csv')
lsource = pd.read_csv(datadir + '/tableA.csv')
rsource = pd.read_csv(datadir + '/tableB.csv')
gt = pd.read_csv(datadir + '/train.csv')
test_df = merge_sources(test, 'ltable_', 'rtable_', lsource, rsource, ['label'], [])[:samples]
train_df = merge_sources(gt, 'ltable_', 'rtable_', lsource, rsource, ['label'], ['id'])
train_noids = train_df.copy().astype(str)

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

In [5]:
train_noids.head()

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV
0,0,C N Red Imperial Red Ale,Redwood Lodge,American Amber / Red Ale,8.10 %,Kinetic Infrared Imperial Red Ale,Kinetic Brewing Company,American Strong Ale,9.30 %
1,1,Sanibel Red Island Ale,Point Ybel Brewing Company,American Amber / Red Ale,5.60 %,Point Ybel Sanibel Red Island Ale,Point Ybel Brewing Company,Irish Ale,5.60 %
2,0,Kniksen India Red Ale,7 Fjell Bryggeri,American Amber / Red Ale,6.80 %,Boneyard Skunkape India Red Ale,Boneyard Beer Company,India Pale Ale &#40; IPA &#41;,6.50 %
3,0,Buffalo Trace Bourbon Barrel Aged Amber Ale,Wolf Hills Brewing Company,American Amber / Red Ale,-,Ballast Point Calico Amber Ale - Rum Barrel Aged,Ballast Point Brewing Company,Amber Ale,5 %
4,1,Poor Richard 's Ale,Dry Dock Brewing Co. .,American Amber / Red Ale,6.50 %,21st Amendment Poor Richards Ale,21st Amendment Brewery,Traditional Ale,7 %


In [6]:
idx = 0
rand_row = test_df.iloc[idx]
l_id = int(rand_row['ltable_id'])
l_tuple = lsource.iloc[l_id]
r_id = int(rand_row['rtable_id'])
r_tuple = rsource.iloc[r_id]

cf_dir = exp_dir + dataset + '/' + model_name + '/' + str(idx)
os.makedirs(cf_dir, exist_ok=True)

label = rand_row["label"]
row_id = str(l_id) + '-' + str(r_id)
item = get_row(l_tuple, r_tuple)
instance = pd.DataFrame(rand_row).transpose().drop(['ltable_id','rtable_id'], axis=1).astype(str)

In [7]:
instance.head()

,label,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV
0,0,Bulleit Bourbon Barrel Aged G'Knight,Oskar Blues Grill & Brew,American Amber / Red Ale,8.70 %,Figure Eight Bourbon Barrel Aged Jumbo Love,Figure Eight Brewing,Barley Wine,-


In [8]:
def predict_fn(x, **kwargs):
    return model.predict(x, **kwargs)

def predict_fn_mojito(x):
    return model.predict(x, mojito=True)

def predict_fn_c(x, **kwargs):
    return model.predict(x, **kwargs)['match_score']

In [9]:
ShapCounterfactual(predict_fn_c, 0.5, train_noids.columns, time_maximum=300).explanation(instance, train_noids)

9/9 [==============================] - 2s 18ms/step


  0%|          | 0/1 [00:00<?, ?it/s]

419/419 [==============================] - 6s 14ms/step


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [53]:
sample = to_token_df(instance.drop(['ltable_id', 'rtable_id'], axis=1))
sample

,ltable_Beer_Name__Bulleit,ltable_Beer_Name__Bourbon,ltable_Beer_Name__Barrel,ltable_Beer_Name__Aged,ltable_Beer_Name__G'Knight,ltable_Brew_Factory_Name__Oskar,ltable_Brew_Factory_Name__Blues,ltable_Brew_Factory_Name__Grill,ltable_Brew_Factory_Name__&,ltable_Brew_Factory_Name__Brew,...,rtable_Beer_Name__Barrel,rtable_Beer_Name__Aged,rtable_Beer_Name__Jumbo,rtable_Beer_Name__Love,rtable_Brew_Factory_Name__Figure,rtable_Brew_Factory_Name__Eight,rtable_Brew_Factory_Name__Brewing,rtable_Style__Barley,rtable_Style__Wine,rtable_ABV__-
0,Bulleit,Bourbon,Barrel,Aged,G'Knight,Oskar,Blues,Grill,&,Brew,...,Barrel,Aged,Jumbo,Love,Figure,Eight,Brewing,Barley,Wine,-


In [54]:
shapc_explainer_token = ShapCounterfactual(predict_fn_c, 0.5, sample, time_maximum=300)

In [22]:
background_data = train_noids.apply(lambda x: ' '.join(x), axis = 1)
background_data

0      0 C N Red Imperial Red Ale Redwood Lodge Ameri...
1      1 Sanibel Red Island Ale Point Ybel Brewing Co...
2      0 Kniksen India Red Ale 7 Fjell Bryggeri Ameri...
3      0 Buffalo Trace Bourbon Barrel Aged Amber Ale ...
4      1 Poor Richard 's Ale Dry Dock Brewing Co. . A...
                             ...                        
263    0 Fresh Hop Red Liquid Mechanics Brewing Co. ....
264    0 Big V Amber Ale FreeWheel Brewing Company Am...
265    0 Red Rose Amber Ale Baird Brewing Company / F...
266    0 Hop Around Imperial Red Ale Big Bay Brewing ...
267    0 Bourbon Barrel Aged Panama Red Ale Flossmoor...
Length: 268, dtype: object

In [56]:
sc_exp = shapc_explainer_token.explanation(sample.values, background_data)

1/1 [==============================] - 0s 21ms/step


InvalidArgumentError: Graph execution error:

Tried to stack list which only contains uninitialized tensors and has a non-fully-defined element_shape: [?,150]
	 [[{{node TensorArrayV2Stack/TensorListStack}}]]
	 [[model_1/Composition/forward_lstm/PartitionedCall_1]] [Op:__inference_predict_function_32458]